In [2]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
inter = pd.read_csv("reliable_data.csv")

Null hypothesis: Non Stationarity exists in the series.

Alternative Hypothesis: Stationarity exists in the series

accepted_keys = where the null hypothesis was accepted 

rejected_keys = where the null hypothesis was rejected 

In [4]:
from statsmodels.tsa.stattools import adfuller

results_dict = {}
accepted_pvalue_keys = []
rejected_pvalue_keys = []
accepted_critical_keys = []
rejected_critical_keys = []

def adf_test(series, key_name):
    result = adfuller(series)
    
    results_dict[key_name] = {
        'Test Statistic': result[0],
        'p-value': result[1],
        'Critical Values': {key: value for key, value in result[4].items()}
    }
    
    if result[1] < 0.05:
        rejected_pvalue_keys.append(key_name)
    else:
        accepted_pvalue_keys.append(key_name)
    
    test_statistic = result[0]
    critical_values = result[4]
    
    if all(test_statistic < crit_value for crit_value in critical_values.values()):
        rejected_critical_keys.append(key_name)
    else:
        accepted_critical_keys.append(key_name)

for key in inter['main_key'].unique():
    series_key = inter[inter['main_key'] == key]['loss_percentage']
    adf_test(series_key, key)


/home/taps/anaconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:940: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


In [5]:
len(accepted_critical_keys)

37

In [6]:
len(accepted_pvalue_keys)

31

Ho(Accepted): There is no trend present in the data.(p-value>0.05)

Ha(Rejected): A trend is present in the data.

In [7]:
import pymannkendall as mk

results_dict_trend = {}
accepted_pvalue_keys_trend = []
rejected_pvalue_keys_trend = []

def mann_kendall(series, key_name):
    result = mk.original_test(series)
    
    results_dict_trend[key_name] = {
        'Trend': result[0],
        'p-value': result[2]
    }
    
    if result[2] < 0.05:
        rejected_pvalue_keys_trend.append(key_name)
    else:
        accepted_pvalue_keys_trend.append(key_name)
    
for key in inter['main_key'].unique():
    series_key = inter[inter['main_key'] == key]['loss_percentage']
    mann_kendall(series_key, key)